# FrontPipe Fast Feed Builder & Tester (v2 full)
Quickly validate a single feed end-to-end:
1) Load your YAML, locate the feed
2) Test subject regex on unread emails in configured folder
3) Save `.msg` as Unicode to cache and move to drop
4) Run orchestrator and show this feed's status

## 0) Settings

In [ ]:
CFG_PATH = 'config/frontpipe.sample.yaml'
MAILBOX_NAME = 'jm@site.com'
FOLDER_PATH  = ['Inbox','Recon Files']
FEED_COUNTERPARTY = 'NT'
FEED_STREAM       = 'EOD_Summary'

## 1) Load YAML and locate feed

In [ ]:
import os, yaml, json, hashlib
CFG_ABS = os.path.abspath(CFG_PATH)
print('USING CONFIG:', CFG_ABS)
with open(CFG_ABS, 'rb') as f: raw = f.read()
print('SHA256:', hashlib.sha256(raw).hexdigest())
cfg = yaml.safe_load(raw)
feed = next(f for f in cfg['feeds'] if f['counterparty']==FEED_COUNTERPARTY and f['stream']==FEED_STREAM)
print(json.dumps(feed, indent=2))

## 2) Regex test on unread emails

In [ ]:
import re, win32com.client
subj_pat = feed.get('expected_patterns',{}).get('subject_regex')
assert subj_pat, 'No subject_regex found.'
subj_re = re.compile(subj_pat)
ns = win32com.client.Dispatch('Outlook.Application').GetNamespace('MAPI')
fld = ns.Folders[feed.get('mailbox', MAILBOX_NAME)]
for p in feed.get('folder_path', FOLDER_PATH): fld = fld.Folders[p]
items = fld.Items.Restrict('[Unread] = true'); items.Sort('[ReceivedTime]', True)
matched_msg = None
for msg in items:
    s = (msg.Subject or '').strip()
    if subj_re.search(s):
        print('✅', s); matched_msg = msg; break
    else:
        print('❌', s)
assert matched_msg, 'No unread emails matched.'

## 3) Save `.msg` (Unicode) to cache and move to drop/Data Files/<date>

In [ ]:
from frontpipe.utils.fs_utils import sanitize_subject, ensure_drop_folder, atomic_move
from frontpipe.utils.date_utils import target_date
from win32com.client import constants
from datetime import datetime, timezone
import os
cache_dir = os.path.join('runtime','inbox_email_cache'); os.makedirs(cache_dir, exist_ok=True)
safe = sanitize_subject(matched_msg.Subject)
cache_path = os.path.join(cache_dir, safe + '.msg')
try:
    matched_msg.SaveAs(cache_path, getattr(constants,'olMSGUnicode',9))
except Exception:
    matched_msg.SaveAs(cache_path, 3)
rule = cfg.get('target_date_rule','prev_bizday_NY')
tgt = target_date(rule, datetime.now(timezone.utc))
drop_dir = ensure_drop_folder('drop', tgt)
final_path = atomic_move(cache_path, drop_dir)
print('Saved & moved to:', os.path.abspath(final_path))
print('Target date:', tgt)

## 4) Run orchestrator for full status

In [ ]:
from frontpipe.orchestrator import run_once
res = run_once(CFG_PATH, '.', now_utc=datetime.now(timezone.utc))
for r in res['master_rows']:
    if r['counterparty']==FEED_COUNTERPARTY and r['stream']==FEED_STREAM:
        print(r['counterparty'], r['stream'], r['status'], r.get('note',''), r.get('saved_path',''))
print('Open runtime/status.html')